In [ ]:
import numpy as np 
import pandas as pd 
!pip install tensorflow_decision_forests --upgrade -q
import tensorflow_decision_forests as tfdf



# From an idea of Faisal Alsrheed :
#https://www.kaggle.com/faisalalsrheed/dancing-with-tensorflow-decision-forests

In [ ]:
!pip install wurlitzer
from wurlitzer import sys_pipes
from sklearn.metrics import log_loss
from sklearn import preprocessing

from tensorflow.keras.metrics import categorical_crossentropy
import tensorflow as tf

import os
import matplotlib.pyplot as plt
import math

In [ ]:

train_df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [ ]:
# Change type because features should be (not sure...) categorical :
train = train_df.iloc[:,1:].astype('object')
test = test_df.iloc[:,1:].astype('object')
train.shape, test.shape

In [ ]:
# target should be num :
le = preprocessing.LabelEncoder()
train["target"]= le.fit_transform(train['target'])
train["target"].head(3)

In [ ]:
# Convert the dataset into a TensorFlow dataset:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="target")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test)

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    validation_ratio = 0.2,
    num_trees=60,
    growing_strategy="BEST_FIRST_GLOBAL",
    max_depth=15,
    categorical_algorithm="RANDOM",
    early_stopping_num_trees_look_ahead = 20,
    subsample = 0.8
    )


In [ ]:
metrics = [tf.keras.metrics.CategoricalCrossentropy(name="categorical_crossentropy")]
#loss = tf.keras.losses.CategoricalCrossentropy(name="categorical_crossentropy")

In [ ]:
model.compile(metrics=["categorical_crossentropy"])


In [ ]:
with sys_pipes():
    model.fit(x=train_ds)


In [ ]:
# Look at the training evolution :

logs = model.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.show()


In [ ]:
pred = model.predict(test_ds)

In [ ]:
# How to get all the parameters description :
?tfdf.keras.GradientBoostedTreesModel


In [ ]:
# Look at the model :
model.summary()

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
submission_df = pd.DataFrame(pred)
submission_df.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4']
submission_df['id'] = submission['id']
submission_df = submission_df[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4']]
submission_df.to_csv("submission_Keras_24.csv", index=False)